# Configuring IP

This example notebook shows how a user can configure the IP address of their board with UDP or FTDI. 

**Refer to *Networking* in the *Advanced* section of the Naluscope manual for assistance with setting up an Ethernet connection to your board.**

**A custom register configuration file (*register-config.yml*) will be generated by the notebook and must be used for future connections to board for as long as the IP configured with this notebook is valid or until the next reboot of the board.**


## Naludaq Version

MinVersion: 0.33.2

In [1]:
# Print Naludaq version 
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

Naludaq version: 0.33.1


## Compatible Boards

* This example applies to board models with USB or Ethernet ports.

## Imports and Variables

In [2]:
from naluconfigs import get_available_models, get_configuration
from naludaq.board import Board, startup_board
from naludaq.communication import ControlRegisters
from naludaq.controllers import get_connection_controller

import os
import yaml

## Logging

The logger setup below will show additional information from the NaluDAQ package. This is useful for debugging purposes, but
may be skipped if not needed.

In [3]:
import logging


def setup_logger(level=logging.INFO):
    """Setup a basic logger."""
    logger = logging.getLogger()
    handler = logging.StreamHandler()
    handler.setFormatter(
        logging.Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s")
    )
    logger.addHandler(handler)
    logger.setLevel(level)
    suppress = [
        "naludaq.UART",
        "naludaq.FTDI",
    ]
    for name in suppress:
        logging.getLogger(name).setLevel(logging.CRITICAL)
    return logger

logger = setup_logger()

try:
    logger.debug("logger already setup")
except:
    logger = setup_logger()

## Options

A list of valid board models can be found using the following code snippet:


In [4]:
# Determining Board Model
print(get_available_models())

['aardvarcv3', 'aardvarcv4', 'aodsv1', 'aodsv2_eval', 'asocv3', 'asocv3s', 'trbhm', 'dsa-c10-8', 'hdsocv1_evalr2', 'hdsocv2_eval', 'hdsocv2_evalr2', 'hiper', 'upac32', 'upac96', 'udc16', 'aodsoc_asoc', 'aodsoc_aods']



The following parameters must be provided to successfully set the IP address:

- **board_model**:       *board_model*
  - Valid options can be found using the command in the previous code block.

- **connection_method**:      *connection_method*
  - This should be "udp" or "ftdi". Other connections methods may be possible but are not supported or implemented in this notebook.

- **register_config_file**: *file_name*
  - The name of the custom register configuration file generated by this notebook. Must be a ".yaml" or ".yml" file. It will contain the default register values with the appropriate modifications for the registers relevant for setting the board's IP address.

- **default_board_addr**: (*ip_address*, *port*)
  - Refer to the Quick Start guide for your board.

- **target_board_addr**:  (*ip_address*, *port*)
  - This is the IP address that you would like to set for the board

- **recv_addr**:          (*ip_address*, *port*)
   - This is the IP address of the computer used to configure the IP address of the board

Set these parameters in the configuration block below. All file and folder names are assumed to be in the same directory as the notebook.

**Note: The code in this notebook is only valid for changing from the board's default IP address to a different IP address and does not support changing to a different IP address from a previous manually configured IP address. In the event that you would like to use a different custom IP address (and your board is not using its default address), you will need to load a custom register configuration file. Future versions of this notebook may include a flag or settings printout to verify configuration.**

**If you would like to avoid loading a custom configuration file or are unable to connect to the board with UDP (i.e. default board address is taken by another device on the network and/or you have no control over network addresses), you can opt to use FTDI or UART connections to configure an IP. These connections are independent of the existing board IP address.**

In [5]:
################
# Board Config #
################
board_model =  "hdsocv2_evalr2"
register_config_file =  "register-config.yml"
connection_method = "udp"

# Validate connection methods
connection_methods = ["udp", "ftdi"]
if connection_method.lower() not in connection_methods:
    print(f"Invalid connection method. Valid connection methods: {connection_methods}")


#############
# IP Config #
#############
target_board_addr = ("192.168.1.32", 4660)
recv_addr = ("192.168.1.198", 4660)

##############
# UDP Config #
##############
default_board_addr = ("192.168.1.60", 4660)

###############
# FTDI Config #
###############
serial_number = "A503VYQF"

Loads the default register and clock configuration.

In [6]:
board = Board(board_model)
board.load_registers()
board.load_clockfile()

Converts *target_ip_addr* into hex codes for applicable registers.

In [7]:
src_ip, src_port = target_board_addr
src_octets = get_connection_controller(board)._get_octets_or_raise(src_ip)


eth_src_addr31_16 = (src_octets[0] << 8) | src_octets[1]

eth_src_addr15_0 = (src_octets[2] << 8) | src_octets[3]

eth_addr_sel = 3
ethernet_src_config = {
    "eth_src_addr31_16": eth_src_addr31_16,
    "eth_src_addr15_0": eth_src_addr15_0,
    "eth_addr_sel": eth_addr_sel
}

Open connection to board. Connection method is specified in the Options section of this notebook.

In [8]:
if connection_method == "udp":
    board.get_udp_connection(default_board_addr, recv_addr)

elif connection_method == "ftdi":
    board.get_ftdi_connection(
        serial_number= serial_number,
        baud=115200
    )

else:
    print(f"{connection_method} is not valid or is not implemented in this notebook.")

Starts the board and writes *target_ip_address* to board. 

In [9]:
with board:
    if startup_board(board):
        ControlRegisters(board).write_many(
            ethernet_src_config
        )
        print(ethernet_src_config)
        print('Successfully configured IP Address')
    else:
        print(f"Failed to start {board_model}")

board.disconnect()


print(f"Disconnected from {board_model}")

2025-08-20 13:10:24,972 naludaq.board                  [INFO  ]: Already connected.
2025-08-20 13:10:25,075 naludaq.board_inits            [INFO  ]: Init board - hdsocv2_evalr2
2025-08-20 13:10:25,077 naludaq.init_hdsocv2           [INFO  ]:  Running: _fpga_init ###########################################################
2025-08-20 13:10:25,442 naludaq.init_hdsocv2           [INFO  ]:  Running: _power_toggle ########################################################
2025-08-20 13:10:25,548 naludaq.init_hdsocv2           [INFO  ]:  Running: _power_toggle ########################################################
2025-08-20 13:10:25,854 naludaq.init_hdsocv2           [INFO  ]:  Running: _program_clock #######################################################
2025-08-20 13:10:25,862 naludaq.controllers.clock.si5341_controller [INFO  ]: Using si5341 config file C:\Users\plegaspi\Documents\nalu-core\naluconfigs\src\naluconfigs\data\clocks\Si5341-RevD-HDSoCv2_31p25-SYSclk_15p625-SSTclk_200-GccClk_

{'eth_src_addr31_16': 49320, 'eth_src_addr15_0': 288, 'eth_addr_sel': 3}
Successfully configured IP Address
Disconnected from hdsocv2_evalr2


## Creating a Register Configuration File

This custom configuration file must be loaded for future connections to the IP address configured using this notebook. Refer to connecting-to-new-ip.ipynb for usage example.

In [10]:
register_config = get_configuration(board_model)
register_config["registers"]["control_registers"]["eth_src_addr31_16"]["value"] = eth_src_addr31_16
register_config["registers"]["control_registers"]["eth_src_addr15_0"]["value"] = eth_src_addr15_0
register_config["registers"]["control_registers"]["eth_addr_sel"]["value"] = eth_addr_sel

with open(register_config_file, "w") as file:
    yaml.dump(register_config, file)

print(f"Saved register configuration to {register_config_file}")

Saved register configuration to register-config.yml


## Connecting to the Board


Loads the custom register configuration and default clock configuration.

In [11]:
board = Board(board_model)
board.load_registers(register_config_file)
board.load_clockfile()

Opens a connection to the board and reads the Ethernet registers related to the board address.

In [12]:
board.get_udp_connection(target_board_addr, recv_addr)

with board:
    if startup_board(board):
        ethernet_registers = [
            "eth_src_addr31_16",
            "eth_src_addr15_0",
            "eth_addr_sel"
        ]

        for register in ethernet_registers:
            print(ControlRegisters(board).read(register))

    else:
        print(f"Failed to start {board_model}")

board.disconnect()

print(f"Disconnected from {board_model}")


2025-08-20 13:10:52,197 naludaq.board                  [INFO  ]: Already connected.
2025-08-20 13:10:52,291 naludaq.board_inits            [INFO  ]: Init board - hdsocv2_evalr2
2025-08-20 13:10:52,291 naludaq.init_hdsocv2           [INFO  ]:  Running: _fpga_init ###########################################################
2025-08-20 13:10:52,657 naludaq.init_hdsocv2           [INFO  ]:  Running: _power_toggle ########################################################
2025-08-20 13:10:52,765 naludaq.init_hdsocv2           [INFO  ]:  Running: _power_toggle ########################################################
2025-08-20 13:10:53,073 naludaq.init_hdsocv2           [INFO  ]:  Running: _program_clock #######################################################
2025-08-20 13:10:53,079 naludaq.controllers.clock.si5341_controller [INFO  ]: Using si5341 config file C:\Users\plegaspi\Documents\nalu-core\naluconfigs\src\naluconfigs\data\clocks\Si5341-RevD-HDSoCv2_31p25-SYSclk_15p625-SSTclk_200-GccClk_

{'address': '2c', 'bitposition': 0, 'bitwidth': 16, 'description': 'upper 16 bits of ethernet source IP address', 'readwrite': 'rw', 'value': 49320}
{'address': '2b', 'bitposition': 0, 'bitwidth': 16, 'description': 'lower 16 bits of ethernet source IP address', 'readwrite': 'rw', 'value': 288}
{'address': '28', 'bitposition': 0, 'bitwidth': 2, 'description': 'enables manually set destination (bit 0) and source (bit 1) IP addresses', 'readwrite': 'rw', 'value': 3}
Disconnected from hdsocv2_evalr2
